<a href="https://colab.research.google.com/github/anilbhatt1/Skunkworks_Mediapipe_Jitter/blob/main/Mediapipe_pose_stabilize_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usage example of MediaPipe Pose Solution API in Python (see also http://solutions.mediapipe.dev/pose).

In [ ]:
# Minimum dependency for MediaPipe Solutions Python API is opencv-python
!pip install opencv-python~=3.4.11
!pip install mediapipe

     |████████████████████████████████| 49.1MB 85kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 34.5MB 123kB/s 


In [ ]:
lm_dict = {0:'nose',
           1:'right_eye_inner', 2:'right_eye', 3:'right_eye_outer',
           4:'left_eye_inner', 5:'left_eye', 6:'left_eye_outer',
           7:'right_ear', 8:'left_ear',
           9:'right_mouth',10:'left_mouth',
           11:'right_shoulder', 12:'left:shoulder',
           13:'right_elbow', 14:'left_elbow',
           15:'right_wrist', 16:'left_wrist',
           17: 'right_pinky', 18:'left_pinky',
           19:'right_index', 20:'left_index',
           21:'right_thumb', 22:'left_thumb',
           23:'right_hip', 24:'left_hip'}

lm_cnt = 25
thresh = 0.03
delta_init = {'nose':[0,0],
              'right_eye_inner':[0,0], 'right_eye':[0,0], 'right_eye_outer':[0,0],
              'left_eye_inner':[0,0],  'left_eye':[0,0],  'left_eye_outer':[0,0],
              'right_ear':[0,0], 'left_ear':[0,0],
              'right_mouth':[0,0], 'left_mouth':[0,0],
              'right_shoulder':[0,0], 'left_shoulder':[0,0],
              'right_elbow':[0,0], 'left_elbow':[0,0],
              'right_wrist':[0,0], 'left_wrist':[0,0],
              'right_pinky':[0,0], 'left_pinky':[0,0],
              'right_index':[0,0], 'left_index':[0,0],
              'right_thumb':[0,0], 'left_thumb':[0,0],
              'right_hip':[0,0], 'left_hip':[0,0]}

frame_cnt = 0
prev_lst, curr_lst            =  [], []
lst = [0,0]
for i in range(lm_cnt):
    prev_lst.append(lst)
    curr_lst.append(lst)
face_pts = 9      # Landmarks 0-8 belongs to face comprising of eyes, nose & ears

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose    = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
#cap = cv2.VideoCapture('/content/9secs_avi.avi')
#cap = cv2.VideoCapture('/content/4secs_avi.avi')
#cap = cv2.VideoCapture('/content/2secs_avi.avi')
cap = cv2.VideoCapture('/content/MostPoses_LR_Full_avi.avi')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('/content/9secs_pose_v12_0.03.avi',fourcc, 20.0, (352,640))
#out = cv2.VideoWriter('/content/4secs_pose_v11_0.03.avi',fourcc, 20.0, (1080,1920))
#out = cv2.VideoWriter('/content/2secs_pose_v11_0.00.avi',fourcc, 20.0, (720,1280))
out = cv2.VideoWriter('/content/MostPoses_LR_Full_v1_0.03.avi',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame_cnt += 1
        image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        delta_dict = delta_init

        # Copying landmarks that we got from model to curr_lst for delta calculation below
        for i in range(lm_cnt):
            lst = [results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y]
            curr_lst[i] = lst
        
        if frame_cnt > 1:

            # Calculating delta value of coordinates for all 25 landmarks for current frame            
            for i in range(lm_cnt): 
                delta                = [ abs(curr_lst[i][0] - prev_lst[i][0]) / prev_lst[i][0], abs(curr_lst[i][1] - prev_lst[i][1]) / prev_lst[i][1] ]
                delta_dict[lm_dict[i]] = delta

            # Checking face. If left_eye-nose or right_eye-nose combination delta stays within threshold, then retain landmarks from previous frame for entire face.    
            left_eye_nose_x  = (delta_dict['nose'][0] < thresh and delta_dict['left_eye'][0] < thresh)
            left_eye_nose_y  = (delta_dict['nose'][1] < thresh and delta_dict['left_eye'][1] < thresh)

            right_eye_nose_x = (delta_dict['nose'][0] < thresh and delta_dict['right_eye'][0] < thresh)
            right_eye_nose_y = (delta_dict['nose'][1] < thresh and delta_dict['right_eye'][1] < thresh)

            if (left_eye_nose_x or right_eye_nose_x):
                for i in range(face_pts):
                    results.pose_landmarks.landmark[i].x  = prev_lst[i][0]

            if (left_eye_nose_y or right_eye_nose_y):
                for i in range(face_pts):
                    results.pose_landmarks.landmark[i].y  = prev_lst[i][1]

            # Checking mouth. If delta value stays within threshold, then retain landmarks from previous frame.
            if delta_dict['right_mouth'][0] < thresh:
                results.pose_landmarks.landmark[9].x  = prev_lst[9][0]   # 9 - right_mouth
            if delta_dict['right_mouth'][1] < thresh:
                results.pose_landmarks.landmark[9].y  = prev_lst[9][1]

            if delta_dict['left_mouth'][0] < thresh:
                results.pose_landmarks.landmark[10].x  = prev_lst[10][0]   # 10 - left_mouth                                                       
            if delta_dict['left_mouth'][1] < thresh:
                results.pose_landmarks.landmark[10].y  = prev_lst[10][1]

            # Checking shoulders. If either right shoulder OR left shoulder delta stays within threshold, then retain landmarks from previous frame for both.
            shoulder_x = (delta_dict['right_shoulder'][0] < thresh or delta_dict['left_shoulder'][0] < thresh)
            shoulder_y = (delta_dict['right_shoulder'][1] < thresh or delta_dict['left_shoulder'][1] < thresh)

            if shoulder_x:
                results.pose_landmarks.landmark[11].x  = prev_lst[11][0]    # 11 - right_shoulder               
                results.pose_landmarks.landmark[12].x  = prev_lst[12][0]    # 12 - left_shoulder
            if shoulder_y:
                results.pose_landmarks.landmark[11].y  = prev_lst[11][1]                  
                results.pose_landmarks.landmark[12].y  = prev_lst[12][1]

            # Checking elbows.If delta value within threshold, then retain landmarks from previous frame.
            if delta_dict['right_elbow'][0] < thresh:
                results.pose_landmarks.landmark[13].x  = prev_lst[13][0]       # 13 - right_elbow
            if delta_dict['right_elbow'][1] < thresh:
                results.pose_landmarks.landmark[13].y  = prev_lst[13][1]      

            if delta_dict['left_elbow'][0] < thresh:
                results.pose_landmarks.landmark[14].x  = prev_lst[14][0]       # 14 - left_elbow     
            if delta_dict['left_elbow'][1] < thresh:
                results.pose_landmarks.landmark[14].y  = prev_lst[14][1]      

            # Checking right_arm. If right_wrist delta remains within threshold value, then retain the landmarks from previous frame for entire right_arm.
            if delta_dict['right_wrist'][0] < thresh:
                results.pose_landmarks.landmark[15].x  = prev_lst[15][0]       # 15 - right_wrist
                results.pose_landmarks.landmark[17].x  = prev_lst[17][0]       # 17 - right_pinky
                results.pose_landmarks.landmark[19].x  = prev_lst[19][0]       # 19 - right_index
                results.pose_landmarks.landmark[21].x  = prev_lst[21][0]       # 21 - right_thumb  

            if delta_dict['right_wrist'][1] < thresh:
                results.pose_landmarks.landmark[15].y  = prev_lst[15][1]       # 15 - right_wrist
                results.pose_landmarks.landmark[17].y  = prev_lst[17][1]       # 17 - right_pinky
                results.pose_landmarks.landmark[19].y  = prev_lst[19][1]       # 19 - right_index
                results.pose_landmarks.landmark[21].y  = prev_lst[21][1]       # 21 - right_thumb

            # Checking left_arm. If left_wrist delta remains within threshold value, then retain the landmarks from previous frame for entire left_arm.
            if delta_dict['left_wrist'][0] < thresh:
                results.pose_landmarks.landmark[16].x  = prev_lst[16][0]       # 16 - left_wrist
                results.pose_landmarks.landmark[18].x  = prev_lst[18][0]       # 18 - left_pinky
                results.pose_landmarks.landmark[20].x  = prev_lst[20][0]       # 20 - left_index
                results.pose_landmarks.landmark[22].x  = prev_lst[22][0]       # 22 - left_thumb  

            if delta_dict['left_wrist'][1] < thresh:
                results.pose_landmarks.landmark[16].y  = prev_lst[16][1]       # 16 - left_wrist
                results.pose_landmarks.landmark[18].y  = prev_lst[18][1]       # 18 - left_pinky
                results.pose_landmarks.landmark[20].y  = prev_lst[20][1]       # 20 - left_index
                results.pose_landmarks.landmark[22].y  = prev_lst[22][1]       # 22 - left_thumb
                                                                                                        
            # Checking Hips. If either right hip OR left hip delta stays within threshold, then retain landmarks from previous frame for both.
            hip_x = (delta_dict['right_hip'][0] < thresh or delta_dict['left_hip'][0] < thresh)
            hip_y = (delta_dict['right_hip'][1] < thresh or delta_dict['left_hip'][1] < thresh)

            if hip_x:
                results.pose_landmarks.landmark[23].x  = prev_lst[23][0]    # 23 - right_hip
                results.pose_landmarks.landmark[24].x  = prev_lst[24][0]    # 24 - left_hip
            if hip_y:
                results.pose_landmarks.landmark[23].y  = prev_lst[23][1]                  
                results.pose_landmarks.landmark[24].y  = prev_lst[24][1]

        # Copying landmarks that we got from model/modified to prev_lst for comparison of next frame
        for i in range(lm_cnt):
            lst = [results.pose_landmarks.landmark[i].x, results.pose_landmarks.landmark[i].y]
            prev_lst[i] = lst

        # Connecting the landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        out.write(image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print('End of Frame # :', frame_cnt)
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

End of Frame # : 941
